In [ ]:
# CNN


In [6]:
# IMPORTS
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers, models
from keras import backend as K

In [7]:
# LOAD DATA
dataframe = pd.read_csv('ML\data\dynamic_pus_using_pus50000_15PUs_201910_2616_25.txt', delimiter=',', header=None)
dataframe_max = pd.read_csv('ML\data\dynamic_pus_max_power50000_15PUs_201910_2616_25.txt', delimiter=',', header=None)

dataframe.reset_index(drop=True, inplace=True)
dataframe_max.reset_index(drop=True, inplace=True)

dataframe_tot = pd.concat([dataframe, dataframe_max.iloc[:, dataframe_max.columns.values[-1]]], axis=1,
                        ignore_index=True)
idx = dataframe_tot[dataframe_tot[dataframe_tot.columns[-1]] == -float('inf')].index
dataframe_tot.drop(idx, inplace=True)

data_reg = np.concatenate((dataframe_tot.values[:, 0:dataframe_tot.shape[1]-3], 
                           dataframe_tot.values[:, dataframe_tot.shape[1]-1:dataframe_tot.shape[1]]), axis=1)
data_class = dataframe_tot.values[:, 0:dataframe_tot.shape[1]-1]
y_class_power = dataframe_tot.values[:, -1]
del dataframe, dataframe_max

In [9]:
# INIT
number_samples = [5] + list(range(10, 101, 10)) + [120, 150, 200, 250, 300, 400, 500, 700] + list(range(1000, 4001, 1000))
number_samples = [10]

In [10]:
def calculate_mu_sigma(data, num_pus):
    sum_non_noise = 0
    for pu_n in range(num_pus): # calculate mu
        sum_non_noise += data[0][pu_n*3+2]
    mu = ((max_x * max_y - num_pus) * noise_floor + sum_non_noise)/(max_x * max_y)
    sum_square = 0
    for pu_n in range(num_pus): # calculate sigma
        sum_square += (data[0][pu_n*3+2]-mu)**2
    sum_square += (max_x * max_y - num_pus) * (noise_floor - mu)**2
    sigma = math.sqrt(sum_square/(max_x * max_y))
    return mu, sigma
# TODO: Consider using min_max normalization becasue difference between values using
# z-score is huge since most of the pixels have the same value, noise floor

In [11]:
## CNN
import math
average_class_diff_power = []
fp_mean_power = []
validation_size, noise_floor = 0.33, -90.0
su_power = 0 # this is not actually su power just a number to show there is an SU in its image
max_x, max_y = 4000, 4000
num_pus = (data_reg.shape[1] - 3)//3
for number_sample in number_samples:
    sample = math.ceil(number_sample * (1 + validation_size))
    x_train, y_train = [], []
    for image_num in range(sample):
        mu, sigma = calculate_mu_sigma(data_reg, num_pus)
        pu_image = [[(noise_floor - mu)/sigma] * max_y for _ in range(max_x)]
        for pu_n in range(num_pus):
            x, y, p = max(0, min(max_x-1, int(data_reg[image_num][pu_n*3]))), max(0, min(max_x-1, int(data_reg[image_num][pu_n*3+1]))), data_reg[image_num][pu_n*3+2]
            pu_image[x][y] = (p - mu)/sigma
        su_image = [[noise_floor] * max_y for _ in range(max_x)]
        su_x, su_y = max(0, min(max_x-1, int(data_reg[image_num][-3]))), max(0, min(max_x-1, int(data_reg[image_num][-2])))
        su_image[su_x][su_y] = su_power
        x_train.append(np.array([pu_image, su_image], dtype='float32'))
        y_train.append(data_reg[image_num][-1])
    x_train = np.concatenate([image[np.newaxis] for image in x_train])
    y_train = np.asarray(y_train, dtype='float32')
    

In [14]:
# TODO, CNN Model here
num_filters = 1
data_format = 'channels_first'
cnn = models.Sequential()
cnn.add(layers.Conv2D(num_filters, (7, 7), activation='relu', input_shape=(2, 4000, 4000), data_format=data_format))
cnn.add(layers.MaxPooling2D(pool_size=(2,2), data_format=data_format))
cnn.add(layers.Conv2D(num_filters, (7, 7), activation='relu', data_format=data_format))
cnn.add(layers.MaxPool2D((2,2), data_format=data_format))
cnn.add(layers.Conv2D(num_filters, (7, 7), activation='relu', data_format=data_format))
cnn.add(layers.MaxPool2D((2,2), data_format=data_format))
cnn.add(layers.Conv2D(num_filters, (7, 7), activation='relu', data_format=data_format))
cnn.add(layers.MaxPool2D((2,2), data_format=data_format))
cnn.add(layers.Flatten())
# cnn.add(layers.Dense(10, activation='sigmoid'))
cnn.add(layers.Dense(1, activation='linear'))

In [15]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 1, 3994, 3994)     99        
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 1, 1997, 1997)     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 1, 1991, 1991)     50        
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 1, 995, 995)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 989, 989)       50        
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 1, 494, 494)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 1, 488, 488)       50        
__________

In [ ]:
cnn.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse','mae'])
cnn.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5


In [177]:
y_train = np.reshape(y_train, (-1,1))

In [179]:
y_train.shape

(14, 1)